# Hive Assignment 2. DML: Find Most Popular Tags
Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:


<code>tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009</code>

Example:

<code>php 5 3 1234 5678</code>

For 'rank' in each year use rank() function. Order by 'rank in 2016'.

The part of the result on the sample dataset:

<code>...
android 3   52  1809    25
php 4   3   1673    215
python  5   11  1585    108
c#  6   1   1519    423
...</code>

Please use the tables 'posts' and 'users' from the database 'stackoverflow_'. 'posts' is partitioned by year and by month. For more details see "Hive assignment. Intro and instructions".

In [1]:
%%writefile describe.hql

USE stackoverflow_;

describe formatted posts;
select * from posts limit 10;

Writing describe.hql


In [2]:
!hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.159 seconds
OK
# col_name            	data_type           	comment             
	 	 
id                  	int                 	                    
post_type_id        	tinyint             	                    
date                	string              	                    
owner_user_id       	int                 	                    
parent_id           	int                 	                    
score               	int                 	                    
favorite_count      	int                 	                    
tags                	array<string>       	                    
	 	 
# Partition Information	 	 
# col_name            	data_type           	comment             
	 	 
year                	string              	                    
month               	string              	                    
	 	 
# Detailed Table Informat

In [14]:
%%writefile task.hql

USE stackoverflow_;

select
    tag,
    rank() over (sort by cnt2016 desc) as rnk2016,
    rank() over (sort by cnt2009 desc) as rnk2009,
    cnt2016,
    cnt2009
from (
    select
        tag,        
        sum(if(year='2016', 1, 0)) as cnt2016,
        sum(if(year='2009', 1, 0)) as cnt2009
    from posts
        lateral view explode(tags) lateral_table as tag
    where post_type_id=1 and year in ('2016','2009')
    group by tag
) as t
order by rnk2016
limit 10;

Overwriting task.hql


In [15]:
!hive -f task.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.184 seconds
Query ID = jovyan_20190504110000_ab910bbe-40f6-4a79-bd2f-10cd3a1baf06
Total jobs = 4
Launching Job 1 out of 4
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1556965619923_0007, Tracking URL = http://cbaa1642b25a:8088/proxy/application_1556965619923_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1556965619923_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2019-05-04 11:00:44,580 Stage-1 map = 0%,  reduce = 0%
2019-05-04 11:00:54,433 Stage-1 map = 100%, 